In [1]:
import requests
import pandas as pd
import yfinance as yf

polygon_api_key = 'vnRBB5lYIL4PMH7VkU72AO19AjtHfenn'

def get_stock_splits(start_date, end_date):
    base_url = 'https://api.polygon.io/v3/reference/splits'
    params = {
        'apiKey': polygon_api_key,
        'start_date': start_date,
        'end_date': end_date,
        'limit': 1000  # Adjust the limit as needed
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        return data.get('results', [])
    except requests.exceptions.RequestException as e:
        print(f"HTTP Request failed: {e}")
        return []

def get_company_name(ticker):
    try:
        stock = yf.Ticker(ticker)
        return stock.info.get('longName', 'Unknown')
    except KeyError:
        return 'Unknown'
    except Exception as e:
        print(f"Failed to get company name for {ticker}: {e}")
        return 'Unknown'

# Example usage
start_date = '2024-01-01'
end_date = '2024-12-31'
splits = get_stock_splits(start_date, end_date)

# Convert the result to a DataFrame for easier handling
df_splits = pd.DataFrame(splits)

# Ensure the 'execution_date' column is in datetime format
df_splits['execution_date'] = pd.to_datetime(df_splits['execution_date'])

# Filter the DataFrame to include only rows within the specified date range
df_splits = df_splits[(df_splits['execution_date'] >= start_date) & (df_splits['execution_date'] <= end_date)]

# Remove rows where split_from is greater than or equal to split_to or split_from is not equal to 1
df_splits = df_splits[(df_splits['split_from'] == 1) & (df_splits['split_from'] < df_splits['split_to'])]

# Add company names to the DataFrame
df_splits['Company Name'] = df_splits['ticker'].apply(get_company_name)

# Format the DataFrame as per the new requirement
df_splits['Date'] = df_splits['execution_date'].dt.strftime('%m/%d/%Y')
df_splits['Symbol'] = df_splits['ticker']
df_splits['Split Ratio'] = df_splits['split_from'].astype(int).astype(str) + ':' + df_splits['split_to'].astype(int).astype(str)

# Select and reorder the columns as required
df_splits = df_splits[['Date', 'Symbol', 'Company Name', 'Split Ratio']]

# Save to a CSV file
if not df_splits.empty:
    df_splits.to_csv('/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/API_Parsed_Data/stock_splits_polygon.csv', index=False)

print(df_splits)


/Users/ardadinc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FIXX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=FIXX&crumb=UQl2mkeuMGp
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/AGVNF?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=AGVNF&crumb=UQl2mkeuMGp
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/WSCO?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomai

           Date Symbol                    Company Name Split Ratio
0    10/09/2024   SONY          Sony Group Corporation         1:5
1    10/03/2024   LRCX        Lam Research Corporation        1:10
2    09/30/2024  KDCLF                         Unknown         1:3
3    09/17/2024   DECK     Deckers Outdoor Corporation         1:6
8    09/12/2024   CTAS              Cintas Corporation         1:4
..          ...    ...                             ...         ...
672  01/10/2024  OMVKY          OMV Aktiengesellschaft         1:4
687  01/05/2024  MICRD          Micron Solutions, Inc.      1:1000
689  01/05/2024  IDKOY         Idemitsu Kosan Co.,Ltd.         1:5
694  01/03/2024  URGYF            Bedford Metals Corp.         1:2
701  01/02/2024  LNGPF  Longfor Group Holdings Limited         1:1

[181 rows x 4 columns]
